### Sentiment Analysis Web App

In [7]:
# Step 1: Import necessary libraries
from transformers import pipeline
import gradio as gr
import matplotlib.pyplot as plt
import io


In [8]:
# Step 2: Load pre-trained Hugging Face pipelines
classifier = pipeline("sentiment-analysis")
classifier_multilingual = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
Device set to use cpu


In [17]:
# Step 3: Core logic for all 6 features
def handle_sentiment(text, file, feature):
    if feature == "Sentiment with Confidence":
        result = classifier(text)
        label = result[0]['label']
        score = result[0]['score']
        return f"Sentiment: {label}, Confidence: {score:.2f}"

    elif feature == "Sentiment with Emoji":
        result = classifier(text)
        label = result[0]['label']
        emoji = "😊" if label == "POSITIVE" else "😢" if label == "NEGATIVE" else "😐"
        return f"Sentiment: {label} {emoji}"

    elif feature == "Sentiment Multilingual":
        result = classifier_multilingual(text)
        return f"Sentiment: {result[0]['label']}"

    elif feature == "Sentiment Over Time":
        sentiment_count = {"POSITIVE": 0, "NEGATIVE": 0, "NEUTRAL": 0}
        for line in text.strip().split("\n"):
            result = classifier(line)
            label = result[0]['label']
            sentiment_count[label] += 1
        return f"POSITIVE: {sentiment_count['POSITIVE']}, NEGATIVE: {sentiment_count['NEGATIVE']}, NEUTRAL: {sentiment_count['NEUTRAL']}"

    elif feature == "Sentiment Visualization":
        sentiment_count = {"POSITIVE": 0, "NEGATIVE": 0, "NEUTRAL": 0}
        for line in text.strip().split("\n"):
            result = classifier(line)
            label = result[0]['label']
            sentiment_count[label] += 1

        # Create a bar chart
        fig, ax = plt.subplots()
        ax.bar(sentiment_count.keys(), sentiment_count.values(), color=['green', 'red', 'gray'])
        ax.set_title("Sentiment Distribution")
        ax.set_ylabel("Count")
        
        import tempfile
        import os
    
        # Save image to a temporary file Gradio can read
        temp_file = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
        plt.savefig(temp_file.name, format="png")
        plt.close()
        return temp_file.name  # Return file path
    

    elif feature == "Sentiment File Upload":
        if file is None:
            return "Please upload a .txt file."
        with open(file.name, "r", encoding="utf-8") as f:
            content = f.read()
        result = classifier(content)
        label = result[0]['label']
        score = result[0]['score']
        return f"Sentiment: {label}, Confidence: {score:.2f}"

    return "Unknown feature."


In [18]:
# Step 4: Define the router that splits text vs image output
def router(text, file, feature):
    result = handle_sentiment(text, file, feature)
    if feature == "Sentiment Visualization":
        return "", result  # No text, return image
    else:
        return result, None  # Text, no image


In [21]:
# Step 5: Build the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Sentiment Analysis Web App")

    feature_choice = gr.Radio(
        ["Sentiment with Confidence", "Sentiment with Emoji", "Sentiment Multilingual", 
         "Sentiment Over Time", "Sentiment Visualization", "Sentiment File Upload"], 
        label="Choose a Feature")

    text_input = gr.Textbox(lines=4, label="Text Input (used for all except file upload)")
    file_input = gr.File(label="Upload .txt File", file_types=[".txt"])

    output_text = gr.Textbox(label="Output")
    output_img = gr.Image(label="Bar Chart (only for Visualization)")

    run_button = gr.Button("Analyze")
    run_button.click(fn=router, inputs=[text_input, file_input, feature_choice], outputs=[output_text, output_img])

# Step 6: Launch the app in the notebook
demo.launch()



* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
